# Laborato 7.2

In [17]:
import nltk
import numpy as np
import regex as re
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
nltk.download('punkt_tab')
stemmer = SnowballStemmer('spanish')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitsu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mitsu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## P3- Matriz de similitudes
> Elabore una matriz de similitud de coseno entre los documentos de la colección "El Señor de los Anillos". Debe aplicar los pesos TF-IDF. 

### 1- Preprocesamiento

In [24]:
import re

with open("stoplist.txt", encoding="latin1") as file:
    stoplist = [line.rstrip().lower() for line in file]
def preprocesamiento(texto,stemming=True):
  words = []
  # 1- convertir a minusculas
  texto = texto.lower()
  # 2- eliminar signos con regex
  texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)
  # 3- tokenizar
  words = nltk.word_tokenize(texto, language='spanish')
  # 3- eliminar stopwords
  words = [word for word in words if word not in stoplist]
  # 4- Aplicar reduccion de palabras (stemming)
  if stemming:        
      words = [stemmer.stem(word) for word in words]
  return words

In [38]:
textos = ["archivo_1.txt", "archivo_2.txt", "archivo_3.txt", "archivo_4.txt", "archivo_5.txt", "archivo_6.txt"]
textos_procesados = []
indice = {}

for file_name in textos:
  file = open('docs2/'+file_name)
  texto = file.read().rstrip()
  texto = preprocesamiento(texto)  
  # print(len(texto))
  textos_procesados.append(texto)

239
240
260
279
241
271


### 2- Similitud de coseno

In [61]:
def compute_tfidf(collection):
  # Calcular los pesos TF-IDF para cada documento de la colección
  N = len(collection)
  
  # Listar todos los términos en la coleccion
  terms = set()
  for doc in collection:
      for word in doc:
          terms.add(word)

  # Calcular la frecuencia de palabras en cada documento usando el set de todas las palabras (terms)
  # tf
  freqs = []
  for doc in collection:
    freq = {}
    # Inicializar todas las frecuencias en 0 para las palabras de 'terms'
    for word in terms:
      freq[word] = 0
    for word in doc:
        if word in freq:
            freq[word] += 1
    freqs.append(freq)

  # Calcular el IDF para cada término
  idfs = {}
  for word in terms:
    idfs[word] = 0  # Inicializar IDF para todos los términos en 0
  for doc in collection:
      unique_words_in_doc = set(doc)  # Solo contar cada palabra una vez por documento
      for word in unique_words_in_doc:
          idfs[word] += 1

  # Calcular IDF para todos los términos
  for word in idfs:
      idfs[word] = np.log(N / idfs[word])

  # Calcular los pesos TF-IDF para cada documento
  tfidf = []
  for i, doc in enumerate(collection):
      doc_tfidf = {word: 0 for word in terms}  # Inicializar TF-IDF con todos los términos
      for word in terms:  # Considerar todos los términos, no solo los que están en el documento
          tf = freqs[i][word]
          doc_tfidf[word] = tf * idfs[word]
      tfidf.append(doc_tfidf)
  
  return tfidf


def cosine_sim(Q, Doc):
    # Obtener los términos comunes entre los dos documentos
    common_terms = set(Q.keys()).intersection(set(Doc.keys()))
    
    # Calcular el producto punto
    dot_product = sum(Q[term] * Doc[term] for term in common_terms)
    
    # Calcular las normas (longitudes) de los vectores
    norm_Q = np.sqrt(sum(value**2 for value in Q.values()))
    norm_Doc = np.sqrt(sum(value**2 for value in Doc.values()))
    
    # Evitar división por cero
    if norm_Q == 0 or norm_Doc == 0:
        return 0.0
    
    # Retornar la similitud de coseno
    return dot_product / (norm_Q * norm_Doc)


  
textos_tfidf = compute_tfidf(textos_procesados)

matriz = []
for doc1 in textos_tfidf:
  row = []
  for doc2 in textos_tfidf:  
    row.append(cosine_sim(doc1, doc2))
  matriz.append(row)

# print(matriz)

# for row in matriz:
#    print(row)


### matriz

|       | Doc1  | Doc2  | Doc3  | Doc4  | Doc5  | Doc6  |
|-------|-------|-------|-------|-------|-------|-------|
| Doc1  | 1.000 | 0.032 | 0.043 | 0.008 | 0.057 | 0.082 |   
| Doc2  | 0.032 | 1.000 | 0.033 | 0.056 | 0.033 | 0.020 |
| Doc3  | 0.043 | 0.033 | 1.000 | 0.085 | 0.025 | 0.092 |
| Doc4  | 0.008 | 0.056 | 0.085 | 1.000 | 0.040 | 0.063 |
| Doc5  | 0.057 | 0.033 | 0.025 | 7.040 | 1.000 | 0.019 |
| Doc6  | 0.082 | 0.020 | 0.092 | 0.063 | 0.019 | 1.000 |

## P4- Similitud de coseno con el Indice Invertido

### 1- Estructura del índice invertido en Python:

In [ ]:
"""
index = {
w1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],
w2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],
w3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],
}

idf = {
w1 : idf_w1,
w2 : idf_w2,
w3 : idf_w3,
}

length ={
doc1: norm_doc1,
doc2: norm_doc2,
doc3: norm_doc3,
...
}
"""

### 2- Algoritmo para construir el índice
#### Paso 1:	Implementar los métodos de la clase

In [59]:
class InvertIndex:
    def __init__(self, index_file):
        self.index_file = index_file
        self.index = {}
        self.idf = {}
        self.length = {}
    
    def building(self, collection_text, position_text):
        # build the inverted index with the collection
        # compute the tf
        # compute the idf
        # compute the length (norm)
        # store in disk
        
        pass
        


        
        
    def retrieval(self, query, k):
        self.load_index()
        # diccionario para el score
        score = {}
        # preprocesar la query: extraer los terminos unicos
        
        # calcular el tf-idf del query
        
        # aplicar similitud de coseno y guardarlo en el diccionario score
                
        # ordenar el score de forma descendente
        result = sorted(score.items(), key= lambda tup: tup[1], reverse=True)
        # retornamos los k documentos mas relevantes (de mayor similitud al query)
        return result[:k] 

    def load_index(self):
        # load index from disk        
        pass   

with open('indice.dat', 'w', encoding="latin1") as f:
    for palabra, doc_ids in sorted(Indice.items()):
        f.write(f"{palabra}: {doc_ids}\n")


### Utilizar el siguiente algoritmo. Asegúrese que la similitud de dos documentos iguales sea 1. 

<img src="image-20241004-152440.png" width="75%" align="" />

### Paso 2:	Probar el Índice

In [3]:
import pandas as pd

# Utilizar la siguiente coleccion de documentos para probar la eficiencia del indice
dataton = pd.read_csv('df_total.csv')
dataton.head()

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra


In [4]:
def mostrarDocumentos(result):
    
    pass

index = InvertIndex("indice.dat")
index.building(dataton, 1) #El texto a procesar esta en la posicion 1

Query1 = "El pais de China y su cooperacion"
result = index.retrieval(Query1, 10)
mostrarDocumentos(result)

# Proponer 3 consultas adiconales

AttributeError: 'InvertIndex' object has no attribute 'load_index'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b824a2a4-ec75-4e0c-9c8d-ae3f8e552746' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>